<a href="https://colab.research.google.com/github/foroughkarandish/Convolutional-Neural-Network-Architecture/blob/master/Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project is not started yet and I am thinking about the best way to explain and implement it. Stay tuned for updates!!!

In this project I will explain CNN and use them to build an image classifier.

**Table of Contents**

1. [Introduction](#1)<br>
2. [Load Packages](#2)<br>
    A. [Import](#21)<br>
    B. [Setup](#22)<br>
    C. [Version](#23)<br>
3. [CNN Architecture](#43)<br>
  A. [Convolutional Layers](#31)<br>
  B. [Edge detection example](#32)<br>
  C. [Padding](#433)<br>
  D. [Strided convolution](#34) <br>
  E. [Convolutions over volumes](#35) <br>
  F. [One Layer of a Convolutional Network](#36) <br>
  G. [A simple convolution network example](#37) <br>
  H. [Pooling layers](#38) <br>
  I. [Convolutional neural network example](#39) <br> 
  J. [Why convolutions?](#399) <br>
5. [Model developement](#5)<br>
    A. [Keras](#51)<br>   
6. [References](#6)<br>

<a id="1"></a>
## 1. References

In deep learning, a convolutional neural network (CNN, or ConvNet) is a class of deep neural networks, most commonly applied to analyzing visual imagery. They are also known as shift invariant or space invariant artificial neural networks (SIANN), based on their shared-weights architecture and translation invariance characteristics. They have applications in image and video recognition, recommender systems, image classification, medical image analysis, natural language processing, and financial time series.<br>


Convolutional networks were inspired by biological processes in that the connectivity pattern between neurons resembles the organization of the animal visual cortex. Individual cortical neurons respond to stimuli only in a restricted region of the visual field known as the receptive field. The receptive fields of different neurons partially overlap such that they cover the entire visual field.<br>


CNNs use relatively little pre-processing compared to other image classification algorithms. This means that the network learns the filters that in traditional algorithms were hand-engineered. This independence from prior knowledge and human effort in feature design is a major advantage [1].<br>

<a id="2"></a>
### A. Load Packages

In [2]:
from sklearn import model_selection, preprocessing, metrics
from sklearn.metrics import mean_squared_error
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from pandas import get_dummies
import lightgbm as lgb
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import sklearn
import warnings

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<a id="22"></a>
### B. Setup

I'm setting up essential visualisation here.

In [3]:
%matplotlib inline
%precision 4
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
pd.set_option("display.precision", 15)

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'

<a id="33"></a>
### C. Version

Checking the versions of my main libraries that I'm going to use in this project.

In [4]:
print('matplotlib: {}'.format(matplotlib.__version__))
print('sklearn: {}'.format(sklearn.__version__))
print('seaborn: {}'.format(sns.__version__))
print('pandas: {}'.format(pd.__version__))
print('numpy: {}'.format(np.__version__))

matplotlib: 3.2.2
sklearn: 0.22.2.post1
seaborn: 0.10.1
pandas: 1.0.5
numpy: 1.18.5


<a id="3"></a>
## 3. CNN Architecture

A convolutional neural network consists of an input and an output layer, as well as multiple hidden layers. The hidden layers of a CNN typically consist of a series of convolutional layers that convolve with a multiplication or other dot product. The activation function is commonly a RELU layer, and is subsequently followed by additional convolutions such as pooling layers, fully connected layers and normalization layers, referred to as hidden layers because their inputs and outputs are masked by the activation function and final convolution.

Though the layers are colloquially referred to as convolutions, this is only by convention. Mathematically, it is technically a sliding dot product or cross-correlation. This has significance for the indices in the matrix, in that it affects how weight is determined at a specific index point[1].

 <a id="31"></a>
### A. Convolutional Layers

<a id="6"></a>
## 6. References

[1]. [wiki](https://en.wikipedia.org/wiki/Convolutional_neural_network)<br>
[2]. 